## CPSC 477 Final Project Part 1: Augmenting ECTSum With Gemini


We used Gemini 1.0 Pro and Google's Generative AI API to augment the summaries in ECTSum. We noticed that the provided bullet-point style summaries in ECTSum were too concise and failed to capture the plethora of useful information in the earnings call transcripts.

For example, this was the provided summary in ECTSum for `SNX_q2_2021.txt`.

```
synnex corporation sees q3 non-gaap earnings per share $1.90 to $2.10.
sees q3 non-gaap earnings per share $1.90 to $2.10.
sees q3 revenue $4.95 billion to $5.45 billion.
qtrly non-gaap diluted earnings per share from continuing operations $2.09.
q2 revenue rose 31 percent to $5.9 billion.
```

To solve this problem, we prompted Gemini to summarize the earnings call transcripts. Furthermore, we emphasized that it should incorporate all the information in the provided summary (which we referred to as the *Key Points* in the prompt). 

Here is the prompt that we used:

```
You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript.

Below is an earnings call transcript. Please summarize this transcript in exactly one paragraph using complete sentences. Keep the summary below 300 words. It is very important that you do not use any titles in the summary. Include relevant information and statistics from the Earnings Call Transcript in your summary. Furthermore, it is very important that you incorporate all the information and statistics from the Key Points and spread it out throughout your summary.

Earnings Call Transcript:
[ect]

Key Points:
[summary]
```

Here is the Gemini-augmented summary for the `SNX_q2_2021.txt` example shown above:

```
Synnex Corporation reported strong Q2 results, exceeding expectations with a 31% year-over-year revenue increase to $5.9 billion. Despite a lower gross margin, non-GAAP diluted earnings per share increased 68% to $2.09. The company's improved cash flow and available liquidity of $3.1 billion demonstrate financial strength. Regarding the proposed merger with Tech Data, regulatory approvals are progressing, and the transaction is expected to close in the second half of 2021. For Q3, Synnex forecasts revenue between $4.95 billion and $5.45 billion, with non-GAAP diluted earnings per share projected in the range of $1.90 to $2.10. While aware of supply chain challenges, the company remains optimistic about IT spending and expects tailwinds from market demand and enterprise project resumption.
```

After generating summaries with Gemini, we scored the summaries in terms of quality and only kept those above our thresholds.

For example, for the `test` fold, we generated 96 summaries. Of those, 96 were kept because they correctly contained only one paragraph. Of those, 85 were kept because the numbers in the generated summary were also present in the full transcript (i.e. no hallucinated numbers). Of those, 50 were kept because they included at least 2 numbers from the *Key Points*. Finally, we added these 50 high-quality summaries to the `test` fold.

|                | train    | val     | test   |
|----------------| -------- | ------- | ------ |
| all            | 336      | 90     | 96    |
| 1 paragraph    | 335      | 90     | 96    |
| 100% precision | 302      | 80     | 85    |
| recall ≥ 2     | 200      | 50      | 50     |

In [1]:
# !pip install google-generativeai
import google.generativeai as genai
import random
from dotenv import load_dotenv
import os
import csv
import pandas as pd
from util import *
env = load_dotenv(override = True)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
genai.configure(api_key = os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-pro")
max_ect_length = 8000

In [6]:
fold = "test"
ect_filenames = os.listdir(f"ectsum/final/{fold}/ects")
gemini_filenames = os.listdir(f"ectsum/final/{fold}/gemini_summaries")
unused_filenames = list(set(ect_filenames).difference(set(gemini_filenames)))
if ".ipynb_checkpoints" in unused_filenames:
    unused_filenames.remove(".ipynb_checkpoints")

unused_filenames = ["AAP_q4_2020.txt"]

entries = []
N = 1 # Number of summaries to generate with Gemini

for filename in unused_filenames:
    ect = ""
    with open(f"ectsum/final/{fold}/ects/{filename}") as f:
        ect = f.read().strip()
    summary = ""
    with open(f"ectsum/final/{fold}/gt_summaries/{filename}") as f:
        summary = f.read().strip()
    entries.append((filename, ect, summary))

# Sort entries in increasing order by transcript length
entries = sorted(entries, key = lambda e: len(e[1]))    

prompt = "You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript.\n\nBelow is an earnings call transcript. Please summarize this transcript in exactly one paragraph using complete sentences. Keep the summary below 300 words. It is very important that you do not use any titles in the summary. Include relevant information and statistics from the Earnings Call Transcript in your summary. Furthermore, it is very important that you incorporate all the information and statistics from the Key Points and spread it out throughout your summary."
for filename, ect, summary in entries:
    if N == 0:
        break
    if os.path.isfile(f"ectsum/final/{fold}/gemini_summaries/{filename}"):
        continue
    key_points = extract_key_points(ect, summary)
    if len(key_points[1]) < 2:
        continue
    ect_length = len(ect)
    if ect_length > max_ect_length:
        ect = f"{ect[:max_ect_length // 2]}...\n...{ect[-max_ect_length // 2:]}"
    response = model.generate_content(f"{prompt}\n\nEarnings Call Transcript:\n{ect}\n\nKey Points:\n{key_points}\n")
    gemini_summary = response.text
    print(ect_length, len(key_points[1]), filename)
    N -= 1
    with open(f"ectsum/final/{fold}/gemini_summaries/{filename}", "w") as f:
        f.write(gemini_summary)
    

18488 3 AAP_q4_2020.txt


In [11]:
fold = "test"
gemini_filenames = os.listdir(f"ectsum/final/{fold}/gemini_summaries")

def is_float(s):
    try: 
        float(s)
        return True
    except: 
        return False

def extract_statistic(s):
    # The string should not be empty
    if len(s) == 0:
        return None
    # It could be a valid number that is not a recent year (i.e 2015-2025)
    if is_float(s) and (float(s) < 2015 or float(s) > 2025):
        return (float(s), None)
    # In the form of $[valid number]
    if s[0] == "$" and is_float(s[1:]):
        return (float(s[1:]), "$")
    # Or in the form of [valid number]%
    if s[-1] == "%" and is_float(s[:-1]):
        return (float(s[:-1]), "%")
    return None

def extract_numbers(summary):
    words = [s for line in summary.split("\n") for s in line.split(" ")]
    words = list(filter(len, words))
    words = list(map(lambda s: s[:-1] if s[-1] in [".", ","] else s, words))
    numbers = set(map(extract_statistic, words))
    numbers.remove(None)
    return list(numbers)

ranking = []
for filename in gemini_filenames:
    if not ".txt" in filename:
        continue
    with open(f"./ectsum/final/{fold}/gemini_summaries/{filename}") as f:
        gemini_summary = f.read().strip()
        paragraph_count = len(list(filter(len, gemini_summary.split("\n"))))
        gemini_numbers = extract_numbers(gemini_summary)
        # print(gemini_numbers)
        with open(f"./ectsum/final/{fold}/gt_summaries/{filename}") as g:
            summary = g.read().strip()
            # print(summary)
            summary_numbers = extract_numbers(summary)
            # print(summary_numbers)
            summary_recall = 0
            for n in summary_numbers:
                if n in gemini_numbers:
                    summary_recall += 1
            with open(f"./ectsum/final/{fold}/ects/{filename}") as h:
                ect = h.read().strip()
                ect_numbers = extract_numbers(ect)
                ect_precision = 0
                for n in gemini_numbers:
                    if n in ect_numbers:
                        ect_precision += 1
                # if filename == "MSI_q3_2021.txt":
                #     print(gemini_numbers, ect_numbers)
                ranking.append((filename, paragraph_count, (summary_recall, len(summary_numbers)), (ect_precision, len(gemini_numbers))))

print(len(ranking))
ranking = list(filter(lambda r: r[1] == 1, ranking))
print("Only keep single paragraph:", len(ranking))
ranking = list(filter(lambda r: r[3][1] > 0 and r[3][0] == r[3][1], ranking))
print("Only keep 100% precision:", len(ranking))
ranking_2 = sorted(ranking, key = lambda r: -r[2][0])
ranking = list(filter(lambda r: r[2][0] >= 2, ranking))
ranking_filenames = list(map(lambda r: r[0], ranking))
# print(ranking_filenames)
print("Only keep recall >= 2:", len(ranking))
for r in ranking_2:
    print(r)

96
Only keep single paragraph: 96
Only keep 100% precision: 85
Only keep recall >= 2: 50
('KFY_q2_2022.txt', 1, (7, 9), (10, 10))
('WST_q1_2020.txt', 1, (5, 8), (8, 8))
('TXT_q1_2021.txt', 1, (5, 11), (16, 16))
('FN_q2_2021.txt', 1, (5, 7), (5, 5))
('NX_q1_2021.txt', 1, (5, 9), (11, 11))
('UVE_q4_2020.txt', 1, (4, 4), (8, 8))
('CHE_q3_2020.txt', 1, (4, 8), (11, 11))
('TXT_q4_2021.txt', 1, (4, 10), (7, 7))
('WST_q3_2020.txt', 1, (4, 8), (4, 4))
('JBL_q1_2022.txt', 1, (4, 11), (9, 9))
('MSI_q3_2021.txt', 1, (4, 10), (9, 9))
('TDY_q4_2020.txt', 1, (4, 4), (7, 7))
('KWR_q1_2020.txt', 1, (4, 9), (12, 12))
('WSO_q2_2021.txt', 1, (3, 4), (11, 11))
('CNC_q3_2021.txt', 1, (3, 14), (3, 3))
('SKT_q1_2021.txt', 1, (3, 3), (8, 8))
('GHL_q4_2020.txt', 1, (3, 4), (5, 5))
('GEF_q3_2021.txt', 1, (3, 3), (10, 10))
('AIT_q2_2020.txt', 1, (3, 9), (3, 3))
('LAZ_q2_2021.txt', 1, (3, 7), (6, 6))
('HII_q1_2021.txt', 1, (3, 3), (6, 6))
('GD_q1_2021.txt', 1, (3, 6), (6, 6))
('PLD_q4_2021.txt', 1, (3, 6), (9, 9)

In [13]:
filenames = {}
filenames["train"] = ['SPG_q4_2020.txt', 'HVT_q2_2021.txt', 'WOR_q3_2022.txt', 'LPX_q2_2021.txt', 'AXL_q3_2021.txt', 'SPG_q4_2021.txt', 'DRI_q1_2022.txt', 'WSO_q3_2020.txt', 'MEI_q4_2021.txt', 'PB_q2_2020.txt', 'WHR_q2_2021.txt', 'PB_q3_2021.txt', 'DRI_q1_2021.txt', 'MTW_q4_2021.txt', 'NX_q2_2021.txt', 'ROL_q4_2021.txt', 'AVA_q2_2020.txt', 'CPF_q2_2021.txt', 'DGX_q3_2021.txt', 'GL_q4_2019.txt', 'WMS_q4_2021.txt', 'ITW_q4_2019.txt', 'RGA_q4_2020.txt', 'MDU_q3_2021.txt', 'WCN_q2_2021.txt', 'MDU_q2_2021.txt', 'TEX_q2_2021.txt', 'BRO_q3_2021.txt', 'VPG_q1_2020.txt', 'WST_q1_2021.txt', 'KFY_q3_2022.txt', 'FN_q1_2022.txt', 'PPL_q1_2021.txt', 'UNF_q2_2021.txt', 'A_q3_2021.txt', 'A_q2_2021.txt', 'MTN_q3_2021.txt', 'PPG_q4_2021.txt', 'COF_q2_2021.txt', 'CHE_q3_2021.txt', 'CHE_q2_2021.txt', 'COO_q4_2020.txt', 'B_q3_2021.txt', 'USB_q2_2021.txt', 'CCK_q1_2021.txt', 'IVC_q3_2020.txt', 'BG_q2_2021.txt', 'NJR_q2_2021.txt', 'MDC_q4_2020.txt', 'WM_q3_2021.txt', 'HSC_q3_2020.txt', 'PAY_q3_2021.txt', 'BKH_q1_2021.txt', 'FE_q3_2021.txt', 'B_q3_2020.txt', 'PAY_q2_2021.txt', 'ESE_q3_2021.txt', 'AAN_q1_2021.txt', 'SRE_q2_2021.txt', 'MHO_q3_2020.txt', 'A_q4_2020.txt', 'SFE_q4_2019.txt', 'IDA_q1_2021.txt', 'GHL_q2_2021.txt', 'WMS_q2_2022.txt', 'UVE_q3_2020.txt', 'CB_q2_2021.txt', 'UVE_q2_2020.txt', 'LRN_q1_2022.txt', 'SNX_q2_2021.txt', 'DHI_q3_2021.txt', 'PPG_q2_2021.txt', 'LII_q1_2021.txt', 'JBL_q2_2021.txt', 'LAZ_q3_2021.txt', 'JBL_q3_2021.txt', 'CBT_q1_2021.txt', 'LEG_q2_2021.txt', 'HLF_q3_2021.txt', 'MAA_q4_2020.txt', 'EXP_q2_2022.txt', 'PSA_q1_2021.txt', 'ICE_q3_2021.txt', 'EAT_q1_2022.txt', 'MEI_q2_2021.txt', 'SPG_q2_2021.txt', 'EGP_q3_2021.txt', 'JBL_q2_2022.txt', 'MTW_q3_2020.txt', 'KAI_q3_2021.txt', 'AME_q3_2020.txt', 'AME_q3_2021.txt', 'AME_q2_2021.txt', 'HSC_q3_2019.txt', 'FLR_q1_2021.txt', 'ENR_q4_2021.txt', 'MEI_q2_2022.txt', 'SXC_q3_2021.txt', 'CTS_q2_2021.txt', 'RES_q1_2021.txt', 'TEX_q1_2021.txt', 'KFY_q1_2022.txt', 'PPL_q3_2020.txt', 'FCF_q3_2021.txt', 'FRO_q1_2021.txt', 'IEX_q3_2021.txt', 'PNW_q1_2020.txt', 'UNF_q1_2021.txt', 'OXM_q4_2019.txt', 'ENV_q2_2021.txt', 'LSI_q1_2021.txt', 'JBT_q2_2021.txt', 'SUN_q2_2021.txt', 'VMI_q3_2021.txt', 'FN_q3_2021.txt', 'AN_q3_2021.txt', 'VMC_q4_2020.txt', 'SKT_q3_2021.txt', 'FNF_q4_2020.txt', 'POR_q1_2021.txt', 'UNF_q1_2022.txt', 'KMB_q4_2020.txt', 'CCL_q1_2022.txt', 'RPT_q3_2020.txt', 'OFC_q1_2021.txt', 'HPP_q2_2021.txt', 'FLR_q4_2020.txt', 'MTB_q1_2021.txt', 'SAM_q3_2020.txt', 'RPT_q3_2021.txt', 'DRI_q2_2021.txt', 'DRI_q3_2021.txt', 'WSO_q1_2021.txt', 'CPT_q2_2020.txt', 'KRG_q4_2021.txt', 'CBT_q4_2021.txt', 'HZO_q4_2021.txt', 'MFA_q1_2020.txt', 'HCA_q1_2021.txt', 'IDA_q4_2019.txt', 'NSP_q2_2021.txt', 'LII_q4_2020.txt', 'NX_q1_2022.txt', 'INN_q3_2021.txt', 'RJF_q3_2021.txt', 'DVA_q2_2021.txt', 'PBH_q1_2022.txt', 'EAT_q3_2021.txt', 'EMN_q4_2020.txt', 'CIM_q2_2020.txt', 'WSR_q3_2021.txt', 'LEG_q1_2021.txt', 'CBT_q2_2021.txt', 'SXT_q3_2020.txt', 'CBT_q3_2021.txt', 'PBI_q3_2021.txt', 'SSD_q2_2020.txt', 'PBI_q2_2021.txt', 'AIN_q1_2021.txt', 'AVY_q3_2021.txt', 'CF_q2_2021.txt', 'VPG_q4_2019.txt', 'DFS_q3_2021.txt', 'SUI_q1_2021.txt', 'CPA_q2_2021.txt', 'DHR_q3_2021.txt', 'DHR_q2_2021.txt', 'HII_q2_2021.txt', 'UVV_q1_2022.txt', 'MEI_q1_2021.txt', 'GD_q2_2021.txt', 'GNE_q1_2020.txt', 'SEM_q1_2021.txt', 'UNH_q3_2021.txt', 'DAR_q1_2021.txt', 'SLB_q4_2021.txt', 'LYV_q1_2021.txt', 'GTY_q3_2021.txt', 'UVE_q1_2021.txt', 'WST_q4_2021.txt', 'TXT_q3_2021.txt', 'WST_q4_2020.txt', 'SNX_q1_2021.txt', 'CTS_q4_2020.txt', 'MED_q2_2021.txt', 'CCK_q3_2021.txt', 'CHD_q3_2020.txt', 'NI_q2_2021.txt', 'SUN_q4_2020.txt', 'SKT_q4_2020.txt', 'FUL_q1_2021.txt', 'AN_q4_2020.txt', 'ALG_q4_2020.txt', 'PLD_q3_2021.txt', 'PLD_q2_2021.txt', 'AZZ_q3_2021.txt', 'CHD_q3_2021.txt', 'SYK_q2_2021.txt', 'DHI_q1_2022.txt', 'SCL_q3_2021.txt']
filenames["val"] = ['HLF_q4_2021.txt', 'ABT_q3_2021.txt', 'CNS_q4_2021.txt', 'AIR_q2_2021.txt', 'EMN_q1_2021.txt', 'RKT_q4_2020.txt', 'ITW_q4_2020.txt', 'NPO_q4_2021.txt', 'FN_q1_2020.txt', 'MDC_q3_2020.txt', 'ROG_q1_2021.txt', 'AN_q1_2021.txt', 'POR_q3_2021.txt', 'UNF_q3_2021.txt', 'USB_q3_2021.txt', 'IVC_q2_2020.txt', 'FUL_q2_2021.txt', 'IQV_q4_2021.txt', 'COO_q4_2021.txt', 'UPS_q3_2021.txt', 'RES_q4_2019.txt', 'RGR_q3_2021.txt', 'RS_q4_2020.txt', 'AN_q3_2020.txt', 'MED_q4_2020.txt', 'EAT_q4_2020.txt', 'RJF_q4_2021.txt', 'CLH_q1_2020.txt', 'NSP_q3_2020.txt', 'WOR_q1_2022.txt', 'CBT_q4_2020.txt', 'EXR_q3_2021.txt', 'RPT_q4_2020.txt', 'LCI_q4_2021.txt', 'KMX_q2_2021.txt', 'WHG_q1_2020.txt', 'FSS_q1_2021.txt', 'CL_q4_2020.txt', 'CPA_q3_2021.txt', 'BDC_q4_2020.txt', 'FTV_q1_2021.txt', 'TNC_q3_2020.txt', 'NUS_q3_2021.txt', 'AFG_q3_2021.txt', 'CMC_q3_2021.txt', 'B_q1_2021.txt', 'NJR_q1_2021.txt', 'FN_q4_2021.txt', 'KEY_q2_2022.txt', 'NNN_q2_2021.txt']
filenames["test"] = ['WSO_q2_2021.txt', 'OFC_q2_2021.txt', 'LIN_q3_2021.txt', 'CNC_q3_2021.txt', 'SKT_q1_2021.txt', 'CHE_q4_2020.txt', 'MBI_q4_2019.txt', 'GHL_q4_2020.txt', 'KFY_q2_2022.txt', 'WST_q1_2020.txt', 'UVE_q4_2020.txt', 'NNN_q1_2021.txt', 'CHE_q3_2020.txt', 'GEF_q3_2021.txt', 'AIT_q2_2020.txt', 'TXT_q1_2021.txt', 'ATR_q1_2021.txt', 'LAZ_q2_2021.txt', 'HZO_q1_2021.txt', 'PBH_q3_2021.txt', 'HII_q1_2021.txt', 'PNW_q4_2019.txt', 'GD_q1_2021.txt', 'HCC_q1_2020.txt', 'CIM_q1_2020.txt', 'SJI_q3_2021.txt', 'FR_q4_2020.txt', 'TXT_q4_2021.txt', 'WST_q3_2020.txt', 'PLD_q4_2021.txt', 'FN_q2_2021.txt', 'NHI_q1_2020.txt', 'LHX_q1_2021.txt', 'HPP_q3_2021.txt', 'LH_q3_2021.txt', 'NX_q1_2021.txt', 'HVT_q1_2021.txt', 'AIR_q1_2022.txt', 'JBL_q1_2022.txt', 'MSI_q3_2021.txt', 'OXM_q2_2021.txt', 'TDY_q4_2020.txt', 'SPG_q1_2021.txt', 'GHM_q2_2021.txt', 'WEC_q4_2021.txt', 'PKG_q4_2020.txt', 'KWR_q1_2020.txt', 'NI_q3_2021.txt', 'KMB_q3_2020.txt', 'FNF_q3_2020.txt']
for fold in filenames:
    for filename in filenames[fold]:
        if not os.path.isfile(f"dataset/{fold}/ects/{filename}"):
            ect = ""
            with open(f"ectsum/final/{fold}/ects/{filename}") as f:
                ect = f.read()
                if len(ect) > max_ect_length:
                    ect = f"{ect[:max_ect_length // 2]}...\n...{ect[-max_ect_length // 2:]}"
            with open(f"dataset/{fold}/ects/{filename}", "w") as f:
                f.write(ect)
        if not os.path.isfile(f"dataset/{fold}/gemini_summaries/{filename}"):
            gemini_summary = ""
            with open(f"ectsum/final/{fold}/gemini_summaries/{filename}") as f:
                gemini_summary = f.read()
            with open(f"dataset/{fold}/gemini_summaries/{filename}", "w") as f:
                f.write(gemini_summary)


In [15]:
filenames = {}
filenames["train"] = os.listdir("dataset/train/gemini_summaries")
filenames["val"] = os.listdir("dataset/val/gemini_summaries")
filenames["test"] = os.listdir("dataset/test/gemini_summaries")
for fold in filenames:
    if ".ipynb_checkpoints" in filenames[fold]:        
        filenames[fold].remove(".ipynb_checkpoints")
        
# Earnings call transcripts:
#   - Train: dataset/train/ects/[filename]
#   - Val: dataset/val/ects/[filename]
#   - Test: dataset/test/ects/[filename]
# Gemini summaries:
#   - Train: dataset/train/gemini_summaries/[filename]
#   - Val: dataset/val/gemini_summaries/[filename]
#   - Test: dataset/test/gemini_summaries/[filename]
print(len(filenames["train"]), len(filenames["val"]), len(filenames["test"]))

prompt_1 = "You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript. Below is an earnings call transcript."
prompt_2 = "Please summarize this transcript in exactly one paragraph using complete sentences. Keep the summary below 300 words. It is very important that you do not use any titles in the summary. Include relevant information and statistics from the Earnings Call Transcript in your summary."
for fold in filenames:
    if not os.path.isfile(f"dataset/{fold}.csv"):
        with open(f"dataset/{fold}.csv", "w") as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames = ["filename", "text"])
            entries = []
            for filename in filenames[fold]:
                entry = {}
                entry["filename"] = filename
                ect = ""
                with open(f"dataset/{fold}/ects/{filename}") as ect_file:
                    ect = ect_file.read().replace("#", "")
                gemini_summary = ""
                with open(f"dataset/{fold}/gemini_summaries/{filename}") as gemini_file:
                    gemini_summary = gemini_file.read().replace("#", "")
                entry["text"] = f"<s>[INST] {prompt_1}\n\nEarnings Call Transcript:\n {ect} {prompt_2} [/INST] # {gemini_summary}</s>" 
                entries.append(entry)
            writer.writeheader()
            writer.writerows(entries)
                

200 50 50


In [25]:
for fold in ["train", "val", "test"]:
    df = pd.read_csv(f"dataset/{fold}.csv")
    print(fold, sorted(list(df["text"].str.len())), "\n") # Prompt + answer lengths

train [5148, 5469, 6398, 6679, 6910, 7044, 7225, 7741, 7869, 7870, 8044, 8102, 8195, 8196, 8227, 8329, 8358, 8367, 8491, 8579, 8607, 8686, 8766, 9010, 9211, 9320, 9324, 9362, 9377, 9390, 9400, 9401, 9406, 9418, 9420, 9422, 9426, 9428, 9434, 9436, 9445, 9447, 9451, 9454, 9456, 9458, 9460, 9463, 9464, 9465, 9465, 9469, 9470, 9476, 9476, 9481, 9482, 9483, 9486, 9487, 9500, 9501, 9503, 9508, 9508, 9510, 9511, 9516, 9517, 9519, 9522, 9524, 9526, 9527, 9531, 9531, 9532, 9533, 9535, 9538, 9542, 9543, 9544, 9548, 9548, 9549, 9553, 9555, 9556, 9556, 9556, 9557, 9559, 9559, 9565, 9566, 9568, 9569, 9574, 9575, 9577, 9578, 9580, 9582, 9582, 9584, 9589, 9589, 9589, 9593, 9594, 9595, 9598, 9601, 9601, 9604, 9604, 9612, 9619, 9619, 9622, 9623, 9626, 9629, 9631, 9633, 9634, 9636, 9638, 9638, 9642, 9645, 9646, 9647, 9648, 9649, 9653, 9655, 9657, 9659, 9659, 9660, 9665, 9667, 9668, 9669, 9672, 9673, 9678, 9679, 9680, 9683, 9684, 9689, 9692, 9694, 9694, 9698, 9698, 9700, 9700, 9702, 9702, 9705, 9706, 970